# Background subtraction using top-hat in scikit-image and pyclesperanto
This notebook compares different implementations of a background subtraction method.

**Note:** benchmarking results vary heavily depending on image size, kernel size, used operations, parameters and used hardware. Use this notebook to adapt it to your use-case scenario and benchmark on your target hardware. If you have different scenarios or use-cases, you are very welcome to submit your notebook as pull-request!

In [1]:
import pyclesperanto_prototype as cle
from skimage import morphology
import time

# to measure kernel execution duration properly, we need to set this flag. It will slow down exection of workflows a bit though
cle.set_wait_for_kernel_finish(True)

# selet a GPU with the following in the name. This will fallback to any other GPU if none with this name is found
cle.select_device('RTX')

<NVIDIA GeForce RTX 2080 Ti on Platform: NVIDIA CUDA (1 refs)>

In [2]:
radius = 10
disk_kernel = morphology.disk(radius)
square_kernel = morphology.square(radius)

In [3]:
# test data
import numpy as np

test_image = np.random.random([1500, 2000])

In [4]:
# top-hat (disk) with scikit-image
result_image = None

for i in range(0, 5):
    start_time = time.time()
    result_image = morphology.white_tophat(test_image, selem=disk_kernel)
    print("skimage top-hat disk duration: " + str(time.time() - start_time))

skimage top-hat disk duration: 2.396149158477783
skimage top-hat disk duration: 2.4051051139831543
skimage top-hat disk duration: 2.4281063079833984
skimage top-hat disk duration: 2.461461067199707
skimage top-hat disk duration: 2.479485511779785


In [5]:
# top-hat (square) with scikit-image
result_image = None

for i in range(0, 5):
    start_time = time.time()
    result_image = morphology.white_tophat(test_image, selem=square_kernel)
    print("skimage top-hat square duration: " + str(time.time() - start_time))

skimage top-hat square duration: 0.144270658493042
skimage top-hat square duration: 0.15311598777770996
skimage top-hat square duration: 0.15148043632507324
skimage top-hat square duration: 0.14820003509521484
skimage top-hat square duration: 0.15033388137817383


In [6]:
# top-hat (disk) with pyclesperanto
result_image = None

test_image_gpu = cle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = cle.top_hat_sphere(test_image_gpu, result_image, radius_x=radius, radius_y=radius)
    print("pyclesperanto top-hat-shere duration: " + str(time.time() - start_time))

pyclesperanto top-hat-shere duration: 0.06879878044128418
pyclesperanto top-hat-shere duration: 0.0225980281829834
pyclesperanto top-hat-shere duration: 0.0
pyclesperanto top-hat-shere duration: 0.015633821487426758
pyclesperanto top-hat-shere duration: 0.015624523162841797


In [7]:
# top-hat (square) using pyclesperanto
result_image = None

test_image_gpu = cle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = cle.top_hat_box(test_image_gpu, result_image, radius_x=radius, radius_y=radius)
    print("pyclesperanto top-hat-box duration: " + str(time.time() - start_time))

pyclesperanto top-hat-box duration: 0.025433778762817383
pyclesperanto top-hat-box duration: 0.015667438507080078
pyclesperanto top-hat-box duration: 0.015633583068847656
pyclesperanto top-hat-box duration: 0.0
pyclesperanto top-hat-box duration: 0.015616178512573242
